<a href="https://colab.research.google.com/github/msdevanms/pyspark_training/blob/main/ETL_training_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup - Spark Installation

In [ ]:
!pip install pyspark

In [ ]:
# Create SparkSession and sparkcontext
from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .master("local")\
                    .appName('Firstprogram')\
                    .getOrCreate()
sc=spark.sparkContext

# Load CSV data

In [ ]:
! wget 'https://raw.githubusercontent.com/msdevanms/pyspark_training/main/data/data_descriptive.csv'

In [ ]:
mydataframe = spark.read.format("csv").option("header","true").load("data_descriptive.csv")

In [ ]:
mydataframe.show()

# Inspect Data

In [ ]:
mydataframe.count()

In [ ]:
mydataframe.printSchema()

In [ ]:
mydataframe.describe().show()

# Setting Schema

In [ ]:
# Setting schema and importing:
from pyspark.sql.types import *
myschema = StructType([
      StructField('id', IntegerType()),
      StructField('first_name', StringType()),
      StructField('last_name', StringType()),
      StructField('gender', StringType()),
      StructField('City', StringType()),
      StructField('JobTitle', StringType()),
      StructField('Salary', StringType()),
      StructField('Latitude', StringType()),
      StructField('Longitude', FloatType())
])
df = spark.read.csv("data_descriptive.csv", header=True, schema = myschema)
df.show()

In [ ]:
df.describe().show()

# Data Cleansing - Null and Duplicate removal

In [ ]:
no_duplicate_df = df.dropDuplicates()

In [ ]:
no_duplicate_df.count()

In [ ]:
no_duplicate_df.sort(no_duplicate_df.id).show()

In [ ]:
# Replace null for 'Unknown':
from pyspark.sql.functions import *
no_null_df = no_duplicate_df.withColumn("clean_city",when(no_duplicate_df.City.isNull(),'Unknown').otherwise(no_duplicate_df.City))

In [ ]:
no_null_df.show(truncate=False)

In [ ]:
no_null_df2 = no_null_df.filter(no_null_df.JobTitle.isNotNull())

In [ ]:
no_null_df3 = no_null_df.filter(no_null_df.JobTitle.isNull())

In [ ]:
no_null_df3.show()

In [ ]:
no_null_df2.count()

998

In [ ]:
no_null_df2.show(truncate=False)

In [ ]:
with_column_df = no_null_df2.withColumn('clean_salary', no_null_df2.Salary.substr(2,100).cast('float'))

In [ ]:
# take(1)[0][0]: take 1st element of df row, column = [0,0]
mean = with_column_df.groupBy().avg('clean_salary').take(1)[0][0]

In [ ]:
mean

In [ ]:
with_column_df.describe().select("summary","Salary", "clean_salary").show()

In [ ]:
from pyspark.sql.functions import lit
with_column_df2 = with_column_df.withColumn('new_salary', when(with_column_df.clean_salary.isNull(), lit(mean)).otherwise(with_column_df.clean_salary))

In [ ]:
with_column_df2.show()

In [ ]:
latitudes = with_column_df2.select("latitude")

In [ ]:
latitudes.show()

In [ ]:
latitudes = latitudes.filter(latitudes.latitude.isNotNull())

In [ ]:
latitudes.show()

In [ ]:
# convert to float:
latitudes = latitudes.withColumn('latitude2', latitudes.latitude.cast('float')).select('latitude2')

In [ ]:
latitudes.show()

In [ ]:
import numpy as np
median = np.median(latitudes.collect())

In [ ]:
print(median)

In [ ]:
# replace null with median:
median_replaced_df = with_column_df2.withColumn('lat', when (with_column_df2.Latitude.isNull(), lit(median)).otherwise(with_column_df2.Latitude))

In [ ]:
median_replaced_df.show()

# Aggregation and Group By

In [ ]:
import pyspark.sql.functions as sqlfunc
genders = median_replaced_df.groupBy('gender').agg(sqlfunc.avg('new_salary').alias('AvgSalary'))

In [ ]:
# Avg salary per gender:
genders.show()

In [ ]:
df = median_replaced_df.withColumn('female_salary', when(median_replaced_df.gender=='Female', median_replaced_df.new_salary).otherwise(lit(0)))

In [ ]:
df.show()

In [ ]:
#lit = literal
df = df.withColumn('male_salary', when(df.gender=='Male', df.new_salary).otherwise(lit(0)))

In [ ]:
df.show()

In [ ]:
# Calculate avg salary per jog title:
df = df.groupBy('JobTitle').agg(sqlfunc.avg('female_salary').alias('female_salary'),sqlfunc.avg('male_salary').alias('male_salary'))

In [ ]:
df.show()

In [ ]:
# difference between gender salary:
df = df.withColumn('delta', df.female_salary - df.male_salary).show()

In [ ]:
cityavg = median_replaced_df.groupBy('City').agg(sqlfunc.avg('new_salary').alias('avgsalary'))

In [ ]:
cityavg = cityavg.sort(col('avgsalary').desc())

In [ ]:
# Top cities with higher avg salary:
cityavg.show()

In [ ]:
df.write.csv('out_dataframe.csv')
df.write.json('out_dataframe.json')
df.write.parquet('out_dataframe.parquet')